### 텍스트 전처리 하기

In [3]:
import re
import numpy as np
import pandas as pd
from konlpy.tag import Okt, Kkma
from tokenizers import Tokenizer
from nltk.tokenize import RegexpTokenizer

In [5]:
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 6.0 MB/s eta 0:00:0000:0100:03
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 7.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 4.6 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 5.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.9/22.9 MB 8.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 6.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 5.2 MB/s eta 0:00:00a 0:00:01
  

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 6.1 MB/s eta 0:00:00
  Created wheel for jax: filename=jax-0.4.9-py3-none-any.whl size=1477708 sha256=788c945e53266010458b67e04224b10acd2518d852363f594972ccd789b1c777
  Stored in directory: /tmp/pip-ephem-wheel-cache-yjk4hwnh/wheels/94/4a/f0/e18a5b2c3845cba6dcf7857deafab68b4a5ccccefeaef0f433
Successfully built jax


In [10]:
from Token import UserTokenizers

from tensorflow.keras.preprocessing.text import Tokenizer as TF_Tokenizer
from tensorflow.keras.utils import to_categorical

In [11]:
ut_cls= UserTokenizers()

In [13]:
traind_df=pd.read_csv("./ratings_train.csv")

In [14]:
test_df=pd.read_csv("./ratings_test.csv")

In [15]:
traind_df.dropna(inplace=True)

In [16]:
test_df.dropna(inplace=True)

### 원핫 인코딩
* 가장 기초적이고 기본적인 방법
* 희소행렬이 만들어짐(희소행렬. 원소 대부분이 0인 벡터)
* 단어집합의 크기가 곧 백터의 차원수가 됨(10000개 -> (_,10000))
* 범주형 데이터 분석에서도 활요한다.(자연어처리 < 범주형)
* 고유값에 해당되는 컬럼만 1, 나머지는 0
* 딥러닝, 데이터 마이닝, 자연어 처리에서도 활용됨

1. pandas 라이브러리에서 원핫 인코딩

In [33]:
fruit_market=pd.DataFrame()
fruit_market['person']=["suji","suji","jenny","jenny","din"]
fruit_market['fruit']=["apple","orange","grape","","banana"]
fruit_market['color']=["red","orange","pupple","red","yellow"]

In [20]:
fruit_market_onehot=pd.get_dummies(fruit_market['fruit'])
fruit_market_onehot

,apple,banana,grape,orange
0,1,0,0,0
1,0,0,0,1
2,0,0,1,0
3,1,0,0,0
4,0,1,0,0


In [27]:
person_market_onehot=pd.get_dummies(fruit_market['person'])
person_market_onehot

,din,jenny,suji
0,0,0,1
1,0,0,1
2,0,1,0
3,0,1,0
4,1,0,0


In [24]:
# n-1 열처리
pd.get_dummies(fruit_market['color'],drop_first=True)

,pupple,red,yellow
0,0,1,0
1,0,0,0
2,1,0,0
3,0,1,0
4,0,0,1


In [26]:
fruit_market_onehot=pd.get_dummies(fruit_market)
fruit_market_onehot.T

,0,1,2,3,4
person_din,0,0,0,0,1
person_jenny,0,0,1,1,0
person_suji,1,1,0,0,0
fruit_apple,1,0,0,1,0
fruit_banana,0,0,0,0,1
fruit_grape,0,0,1,0,0
fruit_orange,0,1,0,0,0
color_orange,0,1,0,0,0
color_pupple,0,0,1,0,0
color_red,1,0,0,1,0


In [28]:
pd.from_dummies(person_market_onehot)

,
0,suji
1,suji
2,jenny
3,jenny
4,din


### scikit-learn을 이용한 원핫인코딩

In [29]:
from sklearn.preprocessing import OneHotEncoder

In [34]:
oh_enc = OneHotEncoder(handle_unknown="ignore")

In [35]:
oh_enc.fit(fruit_market)

OneHotEncoder(handle_unknown='ignore')

In [36]:
## 각각의 카테고리 확인하기
oh_enc.categories_

[array(['din', 'jenny', 'suji'], dtype=object),
 array(['', 'apple', 'banana', 'grape', 'orange'], dtype=object),
 array(['orange', 'pupple', 'red', 'yellow'], dtype=object)]

In [37]:
x_data=oh_enc.transform(fruit_market)

In [38]:
## 데이터 타입 objcet-> numpy.float64
#Sparse Row format
x_data

<5x12 sparse matrix of type '<class 'numpy.float64'>'
	with 15 stored elements in Compressed Sparse Row format>

In [40]:
x_datat_arr=x_data.toarray()
x_datat_arr

array([[0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0.],
       [0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.]])

### 텐서플로우

In [46]:
tf_tokenizer= TF_Tokenizer()
tf_tokenizer.fit_on_texts(traind_df['document'][:5])

In [47]:
print("단어 집합: ",tf_tokenizer.word_index)

단어 집합:  {'아': 1, '더빙': 2, '진짜': 3, '짜증나네요': 4, '목소리': 5, '흠': 6, '포스터보고': 7, '초딩영화줄': 8, '오버연기조차': 9, '가볍지': 10, '않구나': 11, '너무재밓었다그래서보는것을추천한다': 12, '교도소': 13, '이야기구먼': 14, '솔직히': 15, '재미는': 16, '없다': 17, '평점': 18, '조정': 19, '사이몬페그의': 20, '익살스런': 21, '연기가': 22, '돋보였던': 23, '영화': 24, '스파이더맨에서': 25, '늙어보이기만': 26, '했던': 27, '커스틴': 28, '던스트가': 29, '너무나도': 30, '이뻐보였다': 31}


In [48]:
print("단어 집합: ",tf_tokenizer.word_counts)

단어 집합:  OrderedDict([('아', 1), ('더빙', 1), ('진짜', 1), ('짜증나네요', 1), ('목소리', 1), ('흠', 1), ('포스터보고', 1), ('초딩영화줄', 1), ('오버연기조차', 1), ('가볍지', 1), ('않구나', 1), ('너무재밓었다그래서보는것을추천한다', 1), ('교도소', 1), ('이야기구먼', 1), ('솔직히', 1), ('재미는', 1), ('없다', 1), ('평점', 1), ('조정', 1), ('사이몬페그의', 1), ('익살스런', 1), ('연기가', 1), ('돋보였던', 1), ('영화', 1), ('스파이더맨에서', 1), ('늙어보이기만', 1), ('했던', 1), ('커스틴', 1), ('던스트가', 1), ('너무나도', 1), ('이뻐보였다', 1)])


In [52]:
encoded = tf_tokenizer.texts_to_sequences(traind_df['document'][:5])[0]

In [53]:
encoded

[1, 2, 3, 4, 5]

In [55]:
one_hot= to_categorical(encoded)
one_hot

array([[0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]], dtype=float32)

### 카운트 기반의 백터화 - 기본

* Bag of Words(BoW)는 단어를 수치화하는 방법
* 문서 내 단어의 순서와 의미는 고려하지 않음 (-> 고려가 되는 알고리즘 개발 필요)
* 단어의 출현 빈도(frequency)만 고려하여 단어 표현
* 카운트 기반의 단어 표현방법(Count-based Word Representation)으로도 불림 (표현방법 vs 임베딩)

- stop_words: 불용어 적용하는 부분
- tokenizer: 사용자가 만든 토크나이저를 적용할 수 있는 부분
- token_pattern: 정규표현식을 사용자가 작성하여 토큰 기준을 설정 할수 있음
- ngram_range: 튜플형태로 데이터를 입력하여 n-gram단위로 분석할 수잇음


In [56]:
from sklearn.feature_extraction.text import CountVectorizer
import joblib #사이킷 런에서 모델을 저장할때, 사용하도록 권장되는 라이브러리

In [64]:
cnt_vect=CountVectorizer()
cnt_vect.fit(traind_df['document'])

CountVectorizer()

In [65]:
joblib.dump(cnt_vect, './mode_data/cnt_vect.pkl')

['./mode_data/cnt_vect.pkl']

In [63]:
print('vocabulary : ', cnt_vect.vocabulary_)

vocabulary :  {'더빙': 20, '진짜': 59, '짜증나네요': 60, '목소리': 24, '포스터보고': 66, '초딩영화줄': 62, '오버연기조차': 43, '가볍지': 4, '않구나': 35, '너무재밓었다그래서보는것을추천한다': 18, '교도소': 8, '이야기구먼': 49, '솔직히': 31, '재미는': 55, '없다': 38, '평점': 64, '조정': 58, '사이몬페그의': 29, '익살스런': 51, '연기가': 39, '돋보였던': 22, '영화': 42, '스파이더맨에서': 32, '늙어보이기만': 19, '했던': 67, '커스틴': 63, '던스트가': 21, '너무나도': 17, '이뻐보였다': 48, '걸음마': 7, '3세부터': 1, '초등학교': 61, '1학년생인': 0, '8살용영화': 2, 'ㅋㅋㅋ': 3, '별반개도': 27, '아까움': 33, '원작의': 46, '긴장감을': 10, '제대로': 57, '살려내지못했다': 30, '반개도': 25, '아깝다': 34, '욕나온다': 45, '이응경': 50, '길용우': 12, '연기생활이몇년인지': 41, '정말': 56, '발로해도': 26, '그것보단': 9, '낫겟다': 14, '납치': 13, '감금만반복반복': 6, '이드라마는': 47, '가족도없다': 5, '연기못하는사람만모엿네': 40, '액션이': 36, '없는데도': 37, '재미': 54, '있는': 53, '몇안되는': 23, '왜케': 44, '평점이': 65, '낮은건데': 15, '볼만한데': 28, '헐리우드식': 68, '화려함에만': 69, '너무': 16, '길들여져': 11, '있나': 52}


In [66]:
cnt_vect = joblib.load('./mode_data/cnt_vect.pkl')

In [70]:
vec_data= cnt_vect.transform(traind_df['document'])
print("백터화된 데이터 타입 : ",type(vec_data))
print("백터화된 데이터  : ",(vec_data))

##(행의 인덱스, 열의 인덱스) 값

백터화된 데이터 타입 :  <class 'scipy.sparse._csr.csr_matrix'>
백터화된 데이터  :    (0, 71119)	1
  (0, 99567)	1
  (0, 246232)	1
  (0, 248358)	1
  (1, 16352)	1
  (1, 167602)	1
  (1, 190112)	1
  (1, 255126)	1
  (1, 273335)	1
  (2, 57394)	1
  (3, 33783)	1
  (3, 145795)	1
  (3, 177352)	1
  (3, 208071)	1
  (3, 222295)	1
  (3, 234711)	1
  (3, 271982)	1
  (4, 56734)	1
  (4, 63331)	1
  (4, 71680)	1
  (4, 74028)	1
  (4, 133947)	1
  (4, 150442)	1
  (4, 181881)	1
  (4, 185057)	1
  :	:
  (149990, 143710)	1
  (149990, 210829)	1
  (149991, 54243)	1
  (149991, 56643)	1
  (149991, 272521)	1
  (149992, 24486)	1
  (149992, 105397)	1
  (149992, 204171)	1
  (149992, 251014)	1
  (149992, 276532)	1
  (149992, 279753)	1
  (149992, 287413)	1
  (149993, 52347)	1
  (149993, 113562)	1
  (149993, 186872)	1
  (149993, 196356)	1
  (149993, 217551)	1
  (149993, 254425)	1
  (149993, 256303)	1
  (149994, 55879)	1
  (149994, 67785)	1
  (149994, 146244)	1
  (149994, 185057)	2
  (149994, 257229)	1
  (149994, 279282)	1


In [71]:
vec_data.shape

(149995, 293366)

### 모델 훈련에서 모델을 저장하는 타이밍

1. 모델 훈련 (fit)
2. 모델을 저장(dump, save)
3. 훈련된 모델을 다시 데이터를 넣어 벡터화를 진행
4. 저장된 모델을 로드(모델이 잘못 저장된 경우)
5. 테스트 데이터에 저장된 모델 적용하여 벡터화를 진행

** fit_transform: 모델훈련+변환 동시 발생시 유의  **

- train, test : 
train한 모델을 test에 적용해야 하므로 모델을 따로 저장해야함
train_df != test_df (분포, 카테고리(10개/13개)등이 다를수 있음)

1. 컬럼수가 달라진다.
--> 테스트 데이터에 적용을 할 수 없는 경우가 발생함
2. vocab 인덱스가 달라진다
-->분석이 다르게 실시 될수 있음


### 사용자 토크나이저 적용하기

In [72]:
konlpy_M_CNTvect= CountVectorizer(tokenizer=ut_cls.konlyMorphsTokenizer)

In [73]:
konlpy_M_CNTvect.fit(traind_df['document'])

/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


CountVectorizer(tokenizer=<bound method UserTokenizers.konlyMorphsTokenizer of <Token.UserTokenizers object at 0x7f493f3f4a30>>)

In [74]:
konlpy_M_CNTvect_data=konlpy_M_CNTvect.transform(traind_df['document']).toarray()

MemoryError: Unable to allocate 116. GiB for an array with shape (149995, 104160) and data type int64

In [75]:
konlpyN_CNTvect = CountVectorizer(tokenizer=ut_cls.konlpyNounsTokenizer)

In [77]:
konlpyN_CNTvect.fit(traind_df['document'][:100])

CountVectorizer(tokenizer=<bound method UserTokenizers.konlpyNounsTokenizer of <Token.UserTokenizers object at 0x7f493f3f4a30>>)

In [79]:
konlpyN_CNTvect_data=konlpyN_CNTvect.transform(traind_df['document'][:100]).toarray()

In [81]:
konlpyN_CNTvect_data.shape

(100, 535)

In [82]:
konlpyN_CNTvect_data

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

### 모델 개발, 서비스 운영 메모리 에러 발생할때 대처

1. 하드웨어 사양을 향상 시킴
- 메모리 증가 (-> 즉각적인 해결이 않됨, 6개월 이상..?)
- 추가 구매/ 서버 대여
- 클라우드 사용 (비용 지출 : 예산이 확보되어야 함)

2. 소프트웨어
   - 개발 :
    서버 -> 스테이징 서버(백업, 서비스 운영 직전 테스트) -> 운영 서버(실서비스)
   - 메모리 에러가 나는 원인 파악  
     1) 데이터의 행
         - (1~12월까지의 데이터라며 ---???,서비스 uuid/ 행의 제외 불가능
         - 데이터 타입 변경
         - 컬럼 삭제 (feature selection)
      2) 데이터의 열?
         - y값 예측에 영향도가 높은 치쳐만 선택하여 사용
         - 자연어처리부분에서는 인코딩/임베딩 기법을 다른것으로 사용
  
      3) 모델에 저장이 안 된 새로운 데이터
         - 처음 모델을 훈련 할때, 라벨인코딩, 원핫 인코딩... etc로 새로운 데이터가 들어오는 것에 대해 처리
         - etc 데이터를 포함해서 모델을 훈련
         - 만약 모델 훈련 불가 -> 모델 예측결과에서 다른 데이터 전환(비지니스에 따라 따라 다름
         - 실시간 서비스 (모델 훈련, 모델 예측 결과) -> 비효율적
    - 모델 변경
        - 메모리를 많이 사용하는 모델을 제외하고 다른 모델로 분석을 함
        (ex):LSTM, GRU, DeepFM, FM... =>  ML model로 전환
        
        
  2.2. 서비스 운영
   - 앱서비스(추천 서비스, 일배치)에서 에러가 발생 => 방어 로직
     - <모델 결과> 방어로직(예: 오늘 추천이 안 된경우 -> 어제 추천 결과를 한 번더 보여줌)
     - <화면> 방어로직(예: 오늘 추천이 않된 경우 -> 현재 서비스를 복구중입니다.)
     - <화면 정책> 방어로직(추천 결과 10개가 나온다고 해을 떄, 최소 1개(default)만이라도 )
   - 앱서비스에서 에러 방생 -> 방어로직 x
     - 애러 해결 대처 (복구 시간: 에러 원인 확인, 해결, 모델 재수행, 결과 송부)
     - 문제 사항에 대한 문제 작성 (경위서, 시말서)

In [83]:
konlpyN_CNTvect.vocabulary_

{'더빙': 135,
 '목소리': 188,
 '흠': 532,
 '포스터': 498,
 '포스터보고': 499,
 '보고': 230,
 '초': 459,
 '초딩영화줄': 462,
 '딩': 147,
 '영화': 335,
 '줄': 439,
 '오버': 343,
 '오버연기': 344,
 '연기': 327,
 '재': 410,
 '재밓': 414,
 '밓': 202,
 '추천': 467,
 '교도소': 65,
 '이야기': 382,
 '재미': 411,
 '평': 493,
 '평점': 495,
 '점': 423,
 '조정': 432,
 '사이몬페그': 253,
 '익살': 391,
 '스파이': 294,
 '스파이더맨': 295,
 '더': 134,
 '맨': 176,
 '커스틴': 473,
 '던스트': 136,
 '걸음마': 49,
 '3': 10,
 '3세': 12,
 '세': 277,
 '초등학교': 461,
 '1': 1,
 '1학년생인': 8,
 '학년': 511,
 '생인': 266,
 '8': 15,
 '8살용영화': 16,
 '살': 257,
 '용': 351,
 '별': 227,
 '별반개': 228,
 '반개': 209,
 '원작': 360,
 '긴장감': 80,
 '욕': 350,
 '이응': 383,
 '이응경': 384,
 '경': 53,
 '길': 81,
 '우': 354,
 '연기생활': 329,
 '생활': 267,
 '년': 111,
 '발': 212,
 '그것': 70,
 '납': 98,
 '감금': 36,
 '반복': 210,
 '반복반복': 211,
 '이': 376,
 '이드라마': 378,
 '드라마': 143,
 '가족': 34,
 '사람': 251,
 '모': 185,
 '모엿': 187,
 '엿': 333,
 '액션': 318,
 '안': 313,
 '왜케': 349,
 '리': 162,
 '리우드식': 165,
 '우드': 355,
 '식': 299,
 '화려': 527,
 '니트': 125,
 '인피': 39

In [85]:
wp_CNTvect= CountVectorizer(tokenizer=ut_cls.WordPieceTokenizer)

In [86]:
wp_CNTvect.fit(traind_df['document'])

/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


CountVectorizer(tokenizer=<bound method UserTokenizers.WordPieceTokenizer of <Token.UserTokenizers object at 0x7f493f3f4a30>>)

In [88]:
wp_CNTvect_data=wp_CNTvect.transform(traind_df['document'][:100]).toarray()

In [90]:
wp_CNTvect_data.shape

(100, 9873)

In [91]:
wp_CNTvect.vocabulary_

{'아': 6936,
 '더빙': 4963,
 '.': 3509,
 '진짜': 8942,
 '짜증나': 8972,
 '##네요': 651,
 '목소리': 5649,
 '흠': 9837,
 '포스터': 9452,
 '##보고': 1487,
 '초딩': 9103,
 '##영화': 2229,
 '##줄': 2806,
 '오버': 7717,
 '##연기': 2194,
 '##조차': 2751,
 '가볍': 3790,
 '##지': 2846,
 '않': 7140,
 '##구나': 410,
 '[UNK]': 3667,
 '교': 4168,
 '##도': 839,
 '##소': 1748,
 '이야기': 8087,
 '##구먼': 418,
 '솔직히': 6644,
 '재미는': 8414,
 '없다': 7382,
 '평점': 9436,
 '조': 8669,
 '##정': 2708,
 '사이': 6398,
 '##몬': 1342,
 '##페': 3160,
 '##그': 442,
 '##의': 2410,
 '익': 8146,
 '##살': 1663,
 '##스런': 1804,
 '연기가': 7513,
 '돋보': 5020,
 '##였던': 2216,
 '영화': 7587,
 '!': 0,
 '스파이더맨': 6723,
 '##에서': 2168,
 '늙': 4794,
 '##어': 2064,
 '##보이': 1517,
 '##기만': 482,
 '했던': 9673,
 '커': 9229,
 '##스틴': 1836,
 '던': 4970,
 '##스트': 1833,
 '##가': 178,
 '너무나도': 4672,
 '이뻐': 8062,
 '##보': 1485,
 '##였다': 2214,
 '막': 5372,
 '걸': 3993,
 '##음': 2405,
 '##마': 1220,
 '3': 3609,
 '##세': 1728,
 '##부터': 1571,
 '초등학교': 9101,
 '1': 3518,
 '##학년': 3313,
 '##생': 1690,
 '##인': 2493,
 '8': 3

### n-gram

* 등장배경: 통계학적 언어 모델의 한계(희소문제)를 보완하기 위함
* 종류(권장사항) : 1<= n <=4 
* 단점 : 문장 앞부분의 중요한 키워드나 맥락을 놓치는 경우가 발생하기도 함

In [92]:
wp_CNTvect = CountVectorizer(tokenizer=ut_cls.WordPieceTokenizer, ngram_range=(1,2))

In [93]:
wp_CNTvect.fit(traind_df['document'])

/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


CountVectorizer(ngram_range=(1, 2),
                tokenizer=<bound method UserTokenizers.WordPieceTokenizer of <Token.UserTokenizers object at 0x7f493f3f4a30>>)

In [94]:
wp_CNTvect_data=wp_CNTvect.transform(traind_df['document'][:100]).toarray()

In [95]:
wp_CNTvect_data.shape

(100, 832178)

In [96]:
wp_CNTvect.vocabulary_

{'아': 622817,
 '더빙': 500542,
 '.': 375247,
 '진짜': 776174,
 '짜증나': 779797,
 '##네요': 67149,
 '목소리': 541534,
 '아 더빙': 623268,
 '더빙 .': 500645,
 '. .': 375258,
 '. 진짜': 379681,
 '진짜 짜증나': 778149,
 '짜증나 ##네요': 779810,
 '##네요 목소리': 67375,
 '흠': 830528,
 '포스터': 806384,
 '##보고': 159974,
 '초딩': 787934,
 '##영화': 232610,
 '##줄': 302418,
 '오버': 683416,
 '##연기': 230031,
 '##조차': 297526,
 '가볍': 410459,
 '##지': 305011,
 '않': 636735,
 '##구나': 44050,
 '흠 .': 830553,
 '. 포스터': 380109,
 '포스터 ##보고': 806401,
 '##보고 초딩': 160666,
 '초딩 ##영화': 787989,
 '##영화 ##줄': 232980,
 '##줄 .': 302550,
 '. 오버': 378652,
 '오버 ##연기': 683445,
 '##연기 ##조차': 230149,
 '##조차 가볍': 297547,
 '가볍 ##지': 410469,
 '##지 않': 306172,
 '않 ##구나': 636758,
 '[UNK]': 393631,
 '교': 437249,
 '##도': 89184,
 '##소': 186266,
 '이야기': 712648,
 '##구먼': 44555,
 '솔직히': 605074,
 '재미는': 735647,
 '없다': 654755,
 '평점': 804530,
 '조': 756138,
 '##정': 293625,
 '교 ##도': 437262,
 '##도 ##소': 89482,
 '##소 이야기': 186618,
 '이야기 ##구먼': 712664,
 '##구먼 .': 44559,
 '. 솔직히': 

### python 표준 라이브러리 from collections import Counter로도 구현가능

### TF-IDF


* Term Frequency-Inverse Document Frequency
단어의 빈도와 역 문서 빈도(문서의 빈도에 특정 식을 취함)를 사용하여 DTM 내의 각 단어들마다 중요한 정도를 가중치로 주는 방법입니다.

* DTM (Document Term Matrix)내 단어마다 중요도를 고려하여 가중치를 주는 통계적 방법

* 전체 문서에서 골고루 빈출되는 단어의 중요도는 낮다고 판단함
* 특정 문서에서 빈출되는 단어의 중요도는 높다고 함
* 
tf(d,t) : 특정 문서 d에서의 특정 단어 t의 등장 횟수.
df(t) : 특정 단어 t가 등장한 문서의 수.
idf(d, t) : df(t)에 반비례하는 수.

TF-IDF = tf * idf


In [98]:
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

In [99]:
tfidf_vect= TfidfVectorizer()

In [100]:
tfidf_vect.fit(traind_df['document'])

TfidfVectorizer()

In [101]:
joblib.dump(tfidf_vect, './mode_data/tfidf_vect.pkl')

['./mode_data/tfidf_vect.pkl']

In [103]:
x_data=tfidf_vect.transform(traind_df['document'][:100]).toarray()

In [104]:
x_data.shape

(100, 293366)

In [105]:
tfidf_vect.get_feature_names_out()

array(['00', '000', '0000000000000000', ..., '綠林의', '不好', '紐⑦넗濡쒕씪異⑹쟾濡'],
      dtype=object)

In [107]:
traind_df[traind_df['document'].str.contains('紐⑦넗濡쒕씪異⑹쟾濡')]

,Unnamed: 0,id,document,label
77983,77983,8875871,紐⑦넗濡쒕씪異⑹쟾濡,0


### tf-idf 사용자 형태소 분석기 적용하기

In [108]:
bpe_tfidfVect= TfidfVectorizer(tokenizer=ut_cls.WordPieceTokenizer)
bpe_tfidfVect.fit(traind_df['document'])

/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TfidfVectorizer(tokenizer=<bound method UserTokenizers.WordPieceTokenizer of <Token.UserTokenizers object at 0x7f493f3f4a30>>)

In [110]:
bpe_tfidfVect_data=bpe_tfidfVect.transform(traind_df['document'][:500]).toarray()

In [111]:
bpe_tfidfVect_data.shape

(500, 9873)

In [112]:
bpe_tfidfVect.get_feature_names_out()

array(['!', '"', '##0', ..., '힘들었다', '힘을', '힘이'], dtype=object)

In [114]:

kkma_tfidVect = TfidfVectorizer(tokenizer=ut_cls.konlpyNounsTokenizer)
kkma_tfidVect.fit(traind_df['document'][:500])

TfidfVectorizer(tokenizer=<bound method UserTokenizers.konlpyNounsTokenizer of <Token.UserTokenizers object at 0x7f493f3f4a30>>)

In [116]:
import time
import datetime

start = time.time()
kkma_tfidVect_data=kkma_tfidVect.transform(traind_df['document'][:500]).toarray()

end = time.time()

sec = (end - start)
result_list = str(datetime.timedelta(seconds=sec)).split(".")
print(result_list[0])

0:00:10


### 모델이 실행되는 시간을 어떻게 알까?

* 정확한 시간은 다 돌려봐야지 알 수 있다.(컴퓨터, 서버에서 실행되는 것, 인터넷 속도..?)
* 샘플데이터를 훈련시켜 시간을 체크하기

In [118]:
traind_df_0=traind_df[traind_df['label']==0]
traind_df_1=traind_df[traind_df['label']==1]

In [122]:
traind_df_0_select=traind_df_0.sample(1000)
traind_df_1_select=traind_df_1.sample(1000)
train_df_select= pd.concat([traind_df_0_select,traind_df_1_select])
train_df_select.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 75610 to 147157
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  2000 non-null   int64 
 1   id          2000 non-null   int64 
 2   document    2000 non-null   object
 3   label       2000 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 78.1+ KB


In [125]:
bpe_tfidfVect.fit(train_df_select['document'])

/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TfidfVectorizer(tokenizer=<bound method UserTokenizers.WordPieceTokenizer of <Token.UserTokenizers object at 0x7f493f3f4a30>>)

In [126]:
bpe_tfidfVect.get_feature_names_out()

array(['!', '"', '##0', ..., '힘들었다', '힘을', '힘이'], dtype=object)

In [128]:
import pickle

In [134]:
f=open("./mode_data/bpe_tfidfVect_p.pkl","wb")
pickle.dump(bpe_tfidfVect,f)
f.close()

TypeError: 'encdoing' is an invalid keyword argument for dump()

### 분산 표현
* 단어의 의미를 다차원 공간에 백터화 하는 방법
* 단어 백터간 유의임한 유사도를 계산할 수 있음